In [39]:
import os
from src.data_loader import DataLoader
from src.sentiment_analyzer import SentimentAnalyzer
from openai import OpenAI

In [40]:
# Input for OpenAI API key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

In [41]:
data_loader = DataLoader(file_path='data/Reviews.csv')

In [42]:
reviews = data_loader.filter_reviews('B006K2ZZ7K')

Total reviews for product B006K2ZZ7K: 4


In [34]:
reviews

,ProductId,Score,Text,Time
4,B006K2ZZ7K,5,Great taffy at a great price. There was a wid...,1350777600
5,B006K2ZZ7K,4,I got a wild hair for taffy and ordered this f...,1342051200
6,B006K2ZZ7K,5,This saltwater taffy had great flavors and was...,1340150400
7,B006K2ZZ7K,5,This taffy is so good. It is very soft and ch...,1336003200


In [35]:
reviews_text = "\n".join([f"- {text}" for text in reviews['Text']])

In [36]:
reviews_text

"- Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.\n- I got a wild hair for taffy and ordered this five pound bag. The taffy was all very enjoyable with many flavors: watermelon, root beer, melon, peppermint, grape, etc. My only complaint is there was a bit too much red/black licorice-flavored pieces (just not my particular favorites). Between me, my kids, and my husband, this lasted only two weeks! I would recommend this brand of taffy -- it was a delightful treat.\n- This saltwater taffy had great flavors and was very soft and chewy.  Each candy was individually wrapped well.  None of the candies were stuck together, which did happen in the expensive version, Fralinger's.  Would highly recommend this candy!  I served it at a beach-themed party and everyone loved it!\n- This taffy is so good.  It is very soft and chewy.  The flavors are amazing.  I would definitely recommend you buying it.  Ve

In [43]:
prompt = (
    "From the following product reviews, identify the top 3 sentiment categories as a Python list.\n\n"
    "Reviews:\n"
    f"{reviews_text}\n\n"
    "Provide the list in this format:\n"
    '["Category1", "Category2", "Category3"]\n'
    "If fewer than 3 categories are found, provide as many as are available."
)

In [45]:
response = client.chat.completions.create(model="gpt-3.5-turbo",
messages=[
    {"role": "user", "content": prompt}
],
max_tokens=250,
temperature=0.7)

In [46]:
response

ChatCompletion(id='chatcmpl-AMbJm5EDGlSL5E85HhIe6Z8vhT6GI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='["Positive", "Recommendation", "Price"]', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729950030, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=312, total_tokens=322, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

In [47]:
categories_text = response.choices[0].message.content.strip()

In [48]:
categories_text

'["Positive", "Recommendation", "Price"]'

In [49]:
prompt = (
    f"Analyze the sentiment for the category '{category}' in the following product reviews.\n\n"
    "For this category, provide:\n"
    "- Overall sentiment (positive/negative/mixed)\n"
    "- Common themes as a list\n"
    "- An example review\n"
    "- Confidence score as a percentage\n\n"
    "Reviews:\n"
    f"{reviews_text}\n\n"
    "Provide the output in valid JSON format as shown below:\n"
    "{{\n"
    '  "category": "{category}",\n'
    '  "sentiment": "positive/negative/mixed",\n'
    '  "confidence": number,\n'
    '  "themes": ["string"],\n'
    '  "example_review": "string"\n'
    "}}"
)

NameError: name 'category' is not defined

In [ ]:
# app.py
import streamlit as st


def main():
    st.title("Sentiment Analysis Chatbot")
    st.write("Analyze product reviews for sentiment.")


    if not OPENAI_API_KEY:
        st.warning("Please enter your OpenAI API key.")
        st.stop()

    # Initialize the data loader and sentiment analyzer
   
    sentiment_analyzer = SentimentAnalyzer(api_key=OPENAI_API_KEY)

    # Input for product ID
    product_id = st.text_input("Enter Product ID to analyze")

    if st.button("Analyze"):
        if not product_id:
            st.warning("Please enter a Product ID.")
            st.stop()

        with st.spinner(f"Analyzing reviews for {product_id}..."):
            reviews = data_loader.filter_reviews(product_id)

In [53]:
response = client.chat.completions.create(model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=250,
            temperature=0.7)
sentiment_text = response.choices[0].message.content.strip()

In [54]:
sentiment_text

'["Positive", "Positive", "Positive"]'

In [56]:
import json
sentiment_detail = json.loads(sentiment_text)

In [57]:
sentiment_detail

['Positive', 'Positive', 'Positive']